In [1]:
import pandas as pd
#from lxml import etree
import requests
from bs4 import BeautifulSoup

In [7]:
import json
import unicodedata
from datetime import datetime

---- using HTML response ----

In [3]:
import locale
locale.getdefaultlocale()


('en_US', 'UTF-8')

In [4]:
#locale.setlocale(locale.LC_TIME, "sv_SE.UTF-8")
#locale.setlocale(locale.LC_ALL, 'sv_SE')
locale.setlocale(locale.LC_TIME, "sv_SE")
#locale.setlocale(locale.LC_ALL, '')

'sv_SE'

In [43]:
url = "https://www.hemnet.se/salda/bostader"
params= {
    #'housing_form_groups':'apartments',
    'location_ids':'898472',
    'item_types':'bostadsratt',
    'by':'sale_date',
    'order':'asc',
    'preferred_sorting':'true',
    'sold_age':'3m', #1d,4d,1w,1m and so on...
    'page':1,
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
soup = BeautifulSoup(response.content, "html.parser")
response.url

'https://www.hemnet.se/salda/bostader?location_ids=898472&item_types=bostadsratt&by=sale_date&order=asc&preferred_sorting=true&sold_age=3m&page=1'

In [44]:

next_link=soup.find('a','next_page')
while next_link:
    print(next_link['href'])
    next_response = requests.request("GET", "https://www.hemnet.se"+next_link['href'], headers=headers)
    next_soup = BeautifulSoup(next_response.content, "html.parser")
    next_link=next_soup.find('a','next_page')



/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=2&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=3&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=4&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=5&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=6&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=7&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=8&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&item_types=bostadsratt&location_ids=898472&order=asc&page=9&preferred_sorting=true&sold_age=3m
/salda/b

In [5]:
#list to add our objects to
item_list=[]

#find the ul with the results list
result_ul=soup.find('ul','sold-results')

#get the list items with our data (filter out the ads)
result_li=result_ul('li', 'sold-results__normal-hit')

#print(len(result_li))

#start loop...
for item in result_li:
    #a dict to collect all key/values in
    item_dict={}

    #get tracking data json for ids...
    tracking_data=item.attrs['data-tracking-data']
    tracking_data_json=json.loads(tracking_data)

    #add ids
    item_dict['listingId']=tracking_data_json['listingId']
    item_dict['saleId']=tracking_data_json['saleId']
    
    #get the anchor href for the url
    result_li_a=item.find('a','sold-property-link')
    item_dict['url']=result_li_a['href']

    #get our sale listing info
    result_price_info=result_li_a.find('div','sold-property-listing__location')

    #address
    address=result_li_a.find('h2','sold-property-listing__heading').string.strip()
    #address=address.replace(' / ','/') #unicode getting in the way again?
    item_dict['address']=address

    #fee
    fee=result_li_a.find('div','sold-property-listing__fee').string.strip()
    fee=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
    item_dict['fee']=fee

    #living area
    #area=result_li_a.find('div','sold-property-listing__area').string.strip()
    #area=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
    #item_dict['area']=area
    area_el=result_li_a.find('div','sold-property-listing__area')
    if area_el:
        area=area_el.text.strip()
        area_arr="".join(area.split('m²')).split()

        span_el = result_li_a.find('span', 'listing-card__attribute--normal-weight')
        if span_el: 
            total_area=int(area_arr[0])+int(area_arr[2])
            item_dict['living_area']=total_area
            item_dict['rooms']=area_arr[3]
        else:
            item_dict['living_area']=area_arr[0]
            item_dict['rooms']=area_arr[1] 
    
    #get our sale price info
    result_price_info=result_li_a.find('div','sold-property-listing__price-info')

    #ending date
    end_date_el=result_price_info.find('div',class_="sold-property-listing__sold-date")
    if end_date_el:
        end_date_str=end_date_el.string.strip()
        end_date_str=end_date_str.replace('Såld ','')
        end_date=datetime.strptime(end_date_str, '%d %B %Y')
        item_dict['end_date']=end_date

    #end price
    end_price_el=result_price_info.find('div',class_="sold-property-listing__subheading")
    if end_price_el:
        end_price_str=end_price_el.string.strip()
        end_price=int("".join(end_price_str.strip(' kr').replace('Slutpris ','').split()))
        item_dict['end_price']=end_price

    #end price/m2
    end_price_area_el=result_price_info.find('div',class_="sold-property-listing__price-per-m2")
    if end_price_area_el:
        end_price_area_str=end_price_area_el.string.strip()
        end_price_area=int("".join(unicodedata.normalize("NFKD",end_price_area_str).strip('kr/m2').split()))
        item_dict['end_price_area']=end_price_area
    
    #price change
    end_price_change_el=result_price_info.find('div',class_="sold-property-listing__price-change")
    if end_price_change_el:
        end_price_change_str=end_price_change_el.string.strip()
        end_price_change_str="".join(end_price_change_str.strip(' %').split())
        end_price_change=int(end_price_change_str[1:])
        end_price_change_sign=end_price_change_str[0]

        #calculate rate (percentage in float)
        if end_price_change !=0:
            item_dict['end_price_change_rate']=eval(f'100{end_price_change_sign}{end_price_change}')/100
        elif end_price_change==0:
            item_dict['end_price_change_rate']=100/100

        item_dict['end_price_change']=end_price_change
        item_dict['end_price_change_sign']=end_price_change_sign

    item_list.append(item_dict)

#print(item_list)

In [6]:
df=pd.json_normalize(item_list)
df.head(3)

,listingId,saleId,url,address,fee,living_area,rooms,end_date,end_price,end_price_area,end_price_change_rate,end_price_change,end_price_change_sign
0,19078075,8372546871807419582,https://www.hemnet.se/salda/lagenhet-3rum-sode...,"Nackagatan 20, 3 tr",5055,"66,5",3,2022-10-07,5500000,82707,NaN,NaN,NaN
1,19066708,8346866493337241075,"https://www.hemnet.se/salda/lagenhet-1,5rum-so...",Ringvägen 162,1983,38,"1,5",2022-10-07,3000000,78947,1.00,0.0,±
2,19096435,7249123498130730538,https://www.hemnet.se/salda/lagenhet-1rum-sode...,"Flintbacken 2, 12 tr",1498,"29,5",1,2022-10-07,2550000,86441,1.02,2.0,+


import json
import unicodedata
from datetime import datetime


#find the ul with the results list
result_ul=soup.find('ul','sold-results')

#get the list items with our data (filter out the ads)
result_li=result_ul('li', 'sold-results__normal-hit')

print(len(result_li))

#start loop...
for item in result_li:
    
    #get the anchor href for the url
    result_li_a=item.find('a','sold-property-link')

    #living area
    area=result_li_a.find('div','sold-property-listing__area').string.strip()
    area_arr="".join(area.split('m²')).split()
    rooms=area_arr[1]
    living_area=area_arr[0]
    #area=int("".join(unicodedata.normalize("NFKD",area).strip('kr/mån').split()))
    #address
    #address_str=result_price_info.find('div',class_='sold-property-listing__location').string.strip()
    #print(address_str)